In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report, accuracy_score
from sklearn import preprocessing 
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import pandas as pd 
import os
for dirname, _, filenames in os.walk('./kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./kaggle/input/train.csv
./kaggle/input/test.csv
./kaggle/input/sample_solution.csv


In [2]:
path = './kaggle/input/'
trainpath = os.path.join(path,'train.csv')
testpath = os.path.join(path,'test.csv')
X_test = pd.read_csv(testpath)
testid = X_test['id']
df_train = pd.read_csv(trainpath)

In [3]:
vc = df_train['claim'].value_counts().to_frame().reset_index()
vc['percent'] = vc["claim"].apply(lambda x : round(100*float(x) / len(df_train), 2))
vc = vc.rename(columns = {"index" : "Target", "claim" : "Count"})
vc

,Target,Count,percent
0,0,480404,50.15
1,1,477515,49.85


In [4]:
features = df_train.columns[:-1]
target = df_train.columns[-1]
df_train["Missing_Value"] = df_train[features].isna().sum(axis=1) 
df_train["Missing_Value_cat"] = df_train["Missing_Value"].map(lambda x: 0 if x==0 else 1)
X_test["Missing_Value"] = X_test[features].isna().sum(axis=1) 
X_test["Missing_Value_cat"] = X_test["Missing_Value"].map(lambda x: 0 if x==0 else 1)

In [5]:
for column in features:
    avg_val = df_train[column].median()
    df_train[column].fillna(avg_val, inplace=True)
    X_test[column].fillna(avg_val, inplace=True)
features = list(features)
features.extend(['Missing_Value','Missing_Value_cat'])

In [6]:


x = df_train.drop(["claim",], axis=1)
y = df_train["claim"].values

x_scale = preprocessing.MaxAbsScaler().fit_transform(x.values)
x_norm, x_claim = x_scale[y == 0], x_scale[y == 1]

In [7]:
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras import regularizers

In [8]:
## input layer 
input_layer = Input(shape=(x.shape[1],))

## encoding part
encoded = Dense(512, activation='tanh', activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoded = Dense(256, activation='relu')(encoded)
encoded = Dense(128, activation='relu')(encoded)
encoded = Dense(64, activation='relu')(encoded)

decoded = Dense(64, activation = 'tanh')(encoded)
decoded = Dense(128, activation='tanh')(decoded)
decoded = Dense(256, activation='tanh')(decoded)
decoded = Dense(512, activation='tanh')(decoded)


## output layer
output_layer = Dense(x.shape[1], activation='relu')(decoded)

In [9]:
es_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.0002,
    patience=5,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)


In [10]:
autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer="adam", loss=tf.keras.losses.MeanSquaredLogarithmicError(
    reduction="auto", name="mean_squared_logarithmic_error"
))


In [11]:
autoencoder.fit(x_norm, x_norm, 
                batch_size = 512, epochs = 30, 
                shuffle = True, validation_split = 0.20,callbacks=[es_callback]);
autoencoder.fit(x_claim, x_claim, 
                batch_size = 512, epochs = 30, 
                shuffle = True, validation_split = 0.20,callbacks=[es_callback]);

Epoch 1/30
751/751 [==============================] - 11s 14ms/step - loss: 0.0155 - val_loss: 0.0115
Epoch 2/30
751/751 [==============================] - 10s 13ms/step - loss: 0.0124 - val_loss: 0.0115
Epoch 3/30
751/751 [==============================] - 10s 13ms/step - loss: 0.0105 - val_loss: 0.0101
Epoch 4/30
751/751 [==============================] - 10s 14ms/step - loss: 0.0099 - val_loss: 0.0098
Epoch 5/30
751/751 [==============================] - 10s 13ms/step - loss: 0.0097 - val_loss: 0.0097
Epoch 6/30
751/751 [==============================] - 10s 14ms/step - loss: 0.0096 - val_loss: 0.0095
Epoch 7/30
751/751 [==============================] - 10s 14ms/step - loss: 0.0103 - val_loss: 0.0363
Epoch 8/30
751/751 [==============================] - 10s 13ms/step - loss: 0.0147 - val_loss: 0.0112
Epoch 9/30
751/751 [==============================] - 10s 13ms/step - loss: 0.0102 - val_loss: 0.0100
Epoch 10/30
751/751 [==============================] - 10s 13ms/step - loss: 0.009

In [12]:
hidden_representation = Sequential()
n=5
for i in range(n):
    hidden_representation.add(autoencoder.layers[i])  
 

In [13]:
norm_hid_rep = hidden_representation.predict(x_norm)
claim_hid_rep = hidden_representation.predict(x_claim)
rep_x = np.append(norm_hid_rep, claim_hid_rep, axis = 0)
y_n = np.zeros(norm_hid_rep.shape[0])
y_f = np.ones(claim_hid_rep.shape[0])
rep_y = np.append(y_n, y_f)

In [14]:
print(norm_hid_rep)

[[0.29402754 0.         0.         ... 0.08714221 0.20456702 0.3592742 ]
 [0.22711617 0.         0.         ... 0.1784266  0.21856132 0.25152373]
 [0.2220364  0.         0.         ... 0.11515195 0.16723268 0.3078011 ]
 ...
 [0.34775966 0.         0.         ... 0.21388957 0.18677753 0.5731911 ]
 [0.23562789 0.         0.         ... 0.14453909 0.16523984 0.41217864]
 [0.25019422 0.         0.         ... 0.11930601 0.22701208 0.31325635]]


In [15]:
from xgboost import XGBClassifier


model = XGBClassifier(
    max_depth=3,
    subsample=0.5,
    colsample_bytree=0.5,
    n_jobs=-1,
    random_state=5590,
)
model2 = XGBClassifier(
    max_depth=3,
    subsample=0.5,
    colsample_bytree=0.5,
    n_jobs=-1,
    random_state=7993,
)

In [16]:
train_x, val_x, train_y, val_y = train_test_split(rep_x, rep_y, test_size=0.25)
clf = model.fit(train_x, train_y)
clf2 = model2.fit(train_x, train_y)
pred_y = np.floor((clf.predict(val_x)+clf2.predict(val_x))*0.5)

/opt/conda/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:50:22] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:50:55] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [17]:
print ("")
print ("Classification Report: ")
print (classification_report(val_y, pred_y))

print ("")
print ("Accuracy Score: ", accuracy_score(val_y, pred_y))


Classification Report: 
              precision    recall  f1-score   support

         0.0       0.86      0.65      0.74    120114
         1.0       0.72      0.89      0.80    119366

    accuracy                           0.77    239480
   macro avg       0.79      0.77      0.77    239480
weighted avg       0.79      0.77      0.77    239480


Accuracy Score:  0.7719559044596626


In [18]:
X_test_scaled = preprocessing.MaxAbsScaler().fit_transform(X_test)
X_test_rep = hidden_representation.predict(X_test_scaled)

In [21]:
from collections import Counter
# Make predictions
y_pred_test = pd.Series(np.floor((model.predict(X_test_rep)+model2.predict(X_test_rep))*0.5))
y_pred_test = pd.DataFrame(y_pred_test,columns=['claim'])
y_pred_test['id'] = testid
y_pred_test = y_pred_test[['id','claim']]
y_pred_test['claim'] = 0
# Create submission file
y_pred_test.to_csv("zerotrap.csv",index=False)

In [22]:
y_counter = Counter(y_pred_test['claim'])
print(y_counter)

Counter({0: 493474})
